## Lecture 16: Mixture Models: EM Algorithm (MIT Lecture Notes)

This is a companion notebook to tackle the **project** for Unit4. The detailed notes are in **Expectation Maximization.pptx**

**EM Algorithm: GMM (Clustering)**

This is a boiler-plate template to get you guys started on how Expectation and Maximization steps can be implimented using numpy arrays.

Just to emphasize:

**E Step**
- For each point $i$ st $i \in \{1,2,3..N\}$
    - For each. cluster $j$ st $j \in \{1,2,3..K\}$
        - Compute $p(j|i)=\frac{p_jN(x^i,\mu_j,\sigma_jI)}{\sum_{j}p_jN(x^i,\mu_j,sigma_jI)}$
        
**M Step**
- Compute $n_j=\sum^{N}_{i=1}p(j|i)$
- From $n_j$ compute $p_j=\frac{n_j}{N}$
- From $p_j$ and $n_j$ compute $\mu_j = \frac{1}{n_j}\sum^{N}_{i=1}p(j|i)x^i$
- From $p_j$, $n_j$ and $\mu_j$ compute $\sigma_j = \frac{1}{n_j d}\sum^{N}_{i=1}p(j|i)||x^i-\mu_j||^2$

In [1]:
## Python Toy Implimentation EM
import numpy as np
X = np.loadtxt("./data/toy_data.txt")
X.shape

(250, 2)

In [2]:
### Step 1: Randomly select the value of parameter (mu,sigma,pj)
import random
random.seed(42)
mu_1 = X[random.randint(0,250)]
mu_2 = X[random.randint(0,250)]
print(mu_1,mu_2)
mu = np.array([mu_1,mu_2])

[3.806 0.903] [-1.809  1.69 ]


In [3]:
mu[0]

array([3.806, 0.903])

In [4]:
x1=X[1]
p_1 = 1/250
p_2 = 1/250
sigma_1 = 0.45
sigma_2 = sigma_1
mixture = np.array([p_1,p_2])

In [5]:
from scipy import stats
s1 = np.identity(2)*sigma_1**2
s1

array([[0.2025, 0.    ],
       [0.    , 0.2025]])

In [6]:
stats.multivariate_normal.pdf(x1,mean=mu_1,cov=s1)

5.8625252785376755e-52

In [7]:
### Estep
k = 2
post = np.zeros((X.shape[0],k))
ll = 0
for i in range(X.shape[0]):
    for j in range(k):
        likelihood = stats.multivariate_normal.pdf(X[i],mean=mu[j],cov=s1)
        post[i,j] = likelihood*mixture[j]
    total = post[i, :].sum()
    post[i, :] = post[i, :] / total
    ll += np.log(total)

In [8]:
print(post[0:10])

[[2.45529942e-34 1.00000000e+00]
 [4.78320795e-50 1.00000000e+00]
 [3.15151092e-48 1.00000000e+00]
 [2.87265827e-45 1.00000000e+00]
 [7.42177645e-41 1.00000000e+00]
 [2.68275838e-42 1.00000000e+00]
 [2.98155824e-36 1.00000000e+00]
 [4.05552397e-44 1.00000000e+00]
 [1.14696855e-38 1.00000000e+00]
 [1.42271413e-43 1.00000000e+00]]


In [9]:
ll

-6910.840224000402

In [10]:
### MStep
n, d = X.shape
_, K = post.shape

n_hat = post.sum(axis=0)
p = n_hat / n

mu = np.zeros((K, d))
var = np.zeros(K)
for j in range(K):
    # Computing mean
    mu[j, :] = (X * post[:, j, None]).sum(axis=0) / n_hat[j]
    # Computing variance
    sse = ((mu[j] - X)**2).sum(axis=1) @ post[:, j]
    var[j] = sse / (d * n_hat[j])

In [11]:
mu

array([[ 5.43571374,  0.15121951],
       [-2.32260134,  0.85912116]])

In [12]:
var

array([4.35983655, 2.76291311])

In [13]:
p

array([0.43657641, 0.56342359])

**KMeans Using EM**

One can also frame k-means as a special case of EM algorithm.

|EM: GMM|EM:Kmeans|
|--------|---------|
|**E Step:** $p(j,i)$ is computed based on a Gaussian Mixture of cluster centers|**E Step:** We simply find out points which are near the means|
|**M Step:** $\mu$,$\sigma$ and $p_j$ are updated based on $n_j$ obtained from $p(j,i)$ obtained in previous step| **M Step**: Only $\mu$ is updated based on the cluster assignments done previous step|

Below is the code given as part of the project

In [1]:
from typing import Tuple
from common import GaussianMixture, plot, init


def estep(X: np.ndarray, mixture: GaussianMixture) -> np.ndarray:
    """E-step: Assigns each datapoint to the gaussian component with the
    closest mean

    Args:
        X: (n, d) array holding the data
        mixture: the current gaussian mixture

    Returns:
        np.ndarray: (n, K) array holding the soft counts
            for all components for all examples

        """
    n, _ = X.shape
    K, _ = mixture.mu.shape
    post = np.zeros((n, K))

    for i in range(n):
        tiled_vector = np.tile(X[i, :], (K, 1))
        sse = ((tiled_vector - mixture.mu)**2).sum(axis=1)
        j = np.argmin(sse)
        post[i, j] = 1 ## You can notice here we are simply hard counting which points belong to which clusters based on distance,(line 25 and 26) 

    return post


def mstep(X: np.ndarray, post: np.ndarray) -> Tuple[GaussianMixture, float]:
    """M-step: Updates the gaussian mixture. Each cluster
    yields a component mean and variance.

    Args: X: (n, d) array holding the data
        post: (n, K) array holding the soft counts
            for all components for all examples

    Returns:
        GaussianMixture: the new gaussian mixture
        float: the distortion cost for the current assignment
    """
    n, d = X.shape
    _, K = post.shape

    n_hat = post.sum(axis=0)
    p = n_hat / n

    cost = 0
    mu = np.zeros((K, d))
    var = np.zeros(K)

    for j in range(K):
        mu[j, :] = post[:, j] @ X / n_hat[j] ## This step is essentially finding mean of points in each cluster
        sse = ((mu[j] - X)**2).sum(axis=1) @ post[:, j]
        cost += sse
        var[j] = sse / (d * n_hat[j])

    return GaussianMixture(mu, var, p), cost


def run(X: np.ndarray, mixture: GaussianMixture,
        post: np.ndarray) -> Tuple[GaussianMixture, np.ndarray, float]:
    """Runs the mixture model

    Args:
        X: (n, d) array holding the data
        post: (n, K) array holding the soft counts
            for all components for all examples

    Returns:
        GaussianMixture: the new gaussian mixture
        np.ndarray: (n, K) array holding the soft counts
            for all components for all examples
        float: distortion cost of the current assignment
    """

    prev_cost = None
    cost = None
    while (prev_cost is None or prev_cost - cost > 1e-4):
        prev_cost = cost
        post = estep(X, mixture)
        mixture, cost = mstep(X, post)

    return mixture, post, cost


In [2]:
def run_kmeans(X):
    for K in range(1, 5):
        min_cost = None
        best_seed = None
        for seed in range(0, 5):
            mixture, post = init(X, K, seed)
            mixture, post, cost = run(X, mixture, post)
            if min_cost is None or cost < min_cost:
                min_cost = cost
                best_seed = seed

        mixture, post = init(X, K, best_seed)
        mixture, post, cost = run(X, mixture, post)
        print(f'cost: {cost} k:{K}, seed:{seed}')

In [3]:
X = np.loadtxt("./data/toy_data.txt")

In [4]:
run_kmeans(X)

cost: 5462.297452340001 k:1, seed:4
cost: 1684.9079502962372 k:2, seed:4
cost: 1329.5948671544297 k:3, seed:4
cost: 1035.499826539466 k:4, seed:4
